In [1]:
import numpy as np

Reproducing the code from the Karpathy "Let's build the GPT Tokenizer" lecture at https://www.youtube.com/watch?v=zduSFxRajkE.

A good web app for working live with tokenization: https://tiktokenizer.vercel.app.

Got to 16m in the lecture.